In [2]:
from impaintingLib import *
from torch.utils.tensorboard import SummaryWriter

### Visualization :
# - plot_all_img 
# - plot_last_img
# - board_plot_last_img
# - board_loss
# - full_board

### Alteration :
# - squareMask

### Models : 
# - AutoEncoder
# - UNet
# - UNetPartialConv
# - SubPixelNetwork

### Loss : 
# - torch.nn.L1Loss()
# - torch.nn.L2Loss()
# - perceptual_loss
# - totalVariationLoss

torch.cuda.empty_cache()
print("Mémoire allouée : ", torch.cuda.memory_allocated() / 1024**2)
print("Mémoire réservé : ", torch.cuda.memory_reserved() / 1024**2)

# -------------- Parameters

trainloader, testloader = getFaces()

runName   = "AutoEncoder L1 Epo50"
model     = AutoEncoder().to(device)

optimizer  = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.001)
criterions =  [#totalVariationLoss,
               #perceptual_loss,
               torch.nn.L1Loss()]

# -------------- Process

# Train
alterFunc = Alter(min_cut=4, max_cut=60).squareMask
visuFunc  = Visu(runName = runName).board_loss
train(model, optimizer, trainloader, criterions, epochs=50, alter=alterFunc, visu=visuFunc)
model_save(model,runName)

# Test
alterFunc = alterFunc
visuFunc  = Visu(runName = runName).board_plot_last_img
# model     = model_load(model,runName)
test(model, testloader, alter=alterFunc, visu=visuFunc)

# -------------- Display Model

# Tensor Board Model
# writer = SummaryWriter("runs/" + runName)
# example_input, _ = next(iter(trainloader))
# writer.add_graph(model,example_input.cuda())
# writer.close()

Mémoire allouée :  124.1337890625
Mémoire réservé :  180.0


  0%|          | 0/375 [00:00<?, ?it/s]

  0%|          | 0/375 [00:00<?, ?it/s]

  0%|          | 0/375 [00:00<?, ?it/s]

  0%|          | 0/375 [00:00<?, ?it/s]

...

  0%|          | 0/375 [00:00<?, ?it/s]

  0%|          | 0/375 [00:00<?, ?it/s]

  0%|          | 0/375 [00:00<?, ?it/s]

  0%|          | 0/375 [00:00<?, ?it/s]

  0%|          | 0/375 [00:00<?, ?it/s]